In [ ]:


#Cnsq = 10**4.6*baseline_length_AU**-2.15  # using the 96 second data, which evolves less quickly with radius


Cnsq_SI = 1.5*10**6*(10)**-4.05*(baseline_length_AU/10)**-3  #m^-20/3 I kind of made this up based on Armbstrong model in the paper 


print("Cnsq_SI = ", Cnsq_SI)

#to calculate density power sectrum
#Pk = Cnsq*k**-(5/3)  #m^-20/3 power spectrum of the density field (this assumes Komogorov turbulence, but is consistent with observations)

vwind_theyassumed_kmsec = 400 #km/s
beta = 5/3  #this is for consistency currently (11/3 is Kolmogorov -- very different)

#need to check conversion to make sure I understand
Ctsq = Cnsq_SI/(2*np.pi)**beta/(vwind_theyassumed_kmsec*1e3)**(1-beta)  #convert back to power spectrum normalization in frequency (what they really measure but report Cnsq; eqn 7)
Ctsq1em4 = Ctsq*(1e-4)**-beta #to scale to where the observations are, in case want to change beta

print("Ctsq = ", Ctsq, " log10 norm at at nu=1e-4 ", np.log10(Ctsq*(1e-4)**-beta))

#density power in m^-6 /Hz
def get_Pdensity_temporal(nu):
    return Ctsq*nu**-beta #measured to be 1.85+-0.2  (this would be 1.67 if Komogorov)

#density power in m^-6 /Hz -- just so I can scale to different distances
def get_Pdensity_temporal(nu, distance):
    return Ctsq*nu**-beta*(distance/baseline_length_AU)**-3  #measured to be 1.85+-0.2  (this would be 1.67 if Komogorov)


#Let's convert to accelerations
def get_dragsqrtPower_msm2Hzmp5(nuGW, baseline_length_AU):
    Cdrag = 1  #I think this is correct drag coefficient becasue it's ballistic
    #Pdrag = get_Pdensity_temporal(nuGW, baseline_length_AU) 
    Pdrag = 10**logdensityPower(np.log10(nuGW)) #newer drag pwoer
    return Cdrag*(vsolar_wind_kmsec*1e3)**2*1.22*mp_kg*Pdrag**.5*area_spacecraft_drag_msq/mass_spacecraft_kg

acceleration_drag_msm2Hzmp5 = get_dragsqrtPower_msm2Hzmp5(freq_arr, baseline_length_AU)

print("sqrt power from grad = ", get_dragsqrtPower_msm2Hzmp5(1e-5, baseline_length_AU))

make_solarwinddrag_plot = False
if make_solarwinddrag_plot:

    plt.figure(figsize=(10, 6))
    plt.loglog(freq_arr, get_Pdensity_temporal(freq_arr, baseline_length_AU))
    plt.xlabel(r'$\nu$ [Hz]')
    plt.ylabel('Power [m $^{-6}$/$Hz]')
    # Set the y-range
    plt.xlim([1e-5, 1e-3])
    plt.title('Density variations')



    plt.figure(figsize=(10, 6))
    plt.loglog(freq_arr, acceleration_drag_msm2Hzmp5 )
    plt.xlabel(r'$\nu$ [Hz]')
    plt.ylabel('Power [m s$^{-2}$/$Hz]')
    # Set the y-range
    plt.xlim([1e-6, 1e-3])
    plt.title('Accelerations from Density variations')




# Dispersion

In [ ]:
Cnsqincm = Cnsq_SI*100**(-20/3)  #converting to cmsquared


def kappa(nuGHz): 
    return estat**2/( 2*np.pi*me*lightspeed*(10**9*nuGHz)**2)  #coeffient for scattering time;  have \checked that \[Kappa][1] 10^3parsec is 4 seconds*)

#This is the delay from the total column of electrons
def total_delay_dispersion():
    return kappa(nuGHz)*ne_solarwind*baseline_length_AU*AU_in_cm


# this is equation C13 in McQuinn and Boone.  It gives the variance with time given a turbulent spectrum
#currently conversion between 1D and 3D is primative#
def sigmaDispersion(t):
    return 9.4*kappa(nuGHz)*(baseline_length_AU*AU_in_cm)**.5*(vsolar_wind_kmsec*1e5*t)**(5/6)*(Cnsqincm)**(1/2)  #seconds



##to get power I have to regularize integral be integrating from tmin to infinity of |t|^5/3 sin(omega t)
def sigmaDispersiontilde(nuGW, alpha):  #take fourier transform of |t|^5/3
    return sigmaDispersion(1.)*np.sqrt(2.606)/(2*np.pi*nuGW)**(4/3)

def powerSpectrumElectronsCGS(k,alpha):

    if np.abs(alpha -5/3)>.001:
        print("error: only currenlty works for 5/3")
        return 0
    return (2*np.pi)**3*Cnsq_SI*k**(-alpha-2)*100**(-20/3)   #100^(-20/3) converts to CGS and the 2pi^3 is the foureier convention (but need to check)


#This is square root of 
def sigmaDispersiontilde2(nuGW, alpha):
    omega = (2*np.pi*nuGW)
    Ptau = 4/(1+alpha)*kappa(nuGHz)**2*(baseline_length_AU*AU_in_cm)*omega/(vsolar_wind_kmsec*1e5)**2*powerSpectrumElectronsCGS(omega/(vsolar_wind_kmsec*1e5), alpha)

    return np.sqrt(Ptau)

def phaseErrorDisp(nuGW, alpha):
    return  2*np.pi*nuGHz*1e9*sigmaDispersiontilde2(nuGW, alpha)

def herror(nuGW, alpha):
    return np.sqrt(4)*sigmaDispersiontilde2(nuGW, alpha)*lightspeed/(baseline_length_AU*AU_in_cm) #the additional square root of two is because really want twice the difference

def herrorold(nuGW, alpha): 
    return 2*sigmaDispersiontilde(nuGW, alpha)*lightspeed/(baseline_length_AU*AU_in_cm)

alpha =5/3 #beta
print("total delay from dispersion in seconds=", total_delay_dispersion(), total_delay_dispersion()*lightspeed/(baseline_length_AU*AU_in_cm))
print("")

print("total phase from dispersion in sqrt(power) at 1e-6 and 1e-5 Hz=",  1e-6**.5*phaseErrorDisp(1e-6, alpha),  1e-5**.5*phaseErrorDisp(1e-5, alpha))
print("total phase homogeneous ", total_delay_dispersion()*2*np.pi*nuGHz*1e9)
print("")

print("sigma h from dispersion in sqrt(power) at 1e-6 and 1e-5 Hz=", herror(1e-6, alpha), herror(1e-5, alpha),herror(1e-6, alpha)/herror(1e-5, alpha) )

print("old sigma h from dispersion in sqrt(power) at 1e-6 and 1e-5 Hz=", herrorold(1e-6, alpha), herrorold(1e-5, alpha), herrorold(1e-6, alpha)/herrorold(1e-5, alpha))
print("two estimates within factor of 2 now...figure out later difference later")
print()

omega = 1e-5
print(9.4**2*(vsolar_wind_kmsec*1e5)**(5/3)*(2.606)/(omega)**(8/3), 4/(1+alpha)*omega/(vsolar_wind_kmsec*1e5)**2*(2*np.pi)**3*(omega/(vsolar_wind_kmsec*1e5))**(-5/3-2), Cnsq_SI*100**(-20/3), Cnsqincm)

#estimate for correct to this
Rfresnel = np.sqrt(baseline_length_AU*AU_in_cm*lambda0/2*np.pi)  #should really divide by two
correction1 = Rfresnel**2*(1e-5/(vsolar_wind_kmsec*1e5))**2; correction2 = Rfresnel**2*(1e-6/(vsolar_wind_kmsec*1e5))**2; 
print("sigma h from dispersion in sqrt(power) at 1e-6 and 1e-5 Hz=", herror(1e-6, alpha)*correction2, herror(1e-5, alpha)*correction1, correction2, correction1, "Rfresel =", Rfresnel, Rfresnel/(vsolar_wind_kmsec*1e5))

